## Handling RAM-like memory

- Definition of afixed size memory support

In [1]:
MEMSIZE = 1000 # total size of memory
mem = [None for _ in range(MEMSIZE)] # memory
stack_pos = 0 # head to first free position in stack
heap_pos = MEMSIZE # head to last occupied position in heap

- Initialization of the memory

In [2]:
def mem_init():
    global mem, stack_pos, heap_pos
    mem = [None for _ in range(MEMSIZE)] # memory
    stack_pos = 0 # head to first free position in stack
    heap_pos = MEMSIZE # head to last occupied position in heap

- Visuzlize memory content (i.e., [stck0, stack1, stack2, ..., heaph2, heap1, heap0])

In [3]:
def mem_dump(sigma):
    print("sigma = ", sigma, " mem = ", 
          [ mem[i] for i in range(stack_pos)] + ["..."] + 
           [ mem[i] for i in range(heap_pos, MEMSIZE)])

- Handling heap memory allocation (*malloc*) and release (*free*)

In [4]:
def malloc(npositions):
    global heap_pos
    heap_pos -= npositions
    return heap_pos

In [5]:
def free(address):
    pass # no action here, garbage collection is a complex task

## Expression and Boolean Interpreter

Let's review the Expression and Boolean parser from last week

In [6]:
from lark import Lark, InlineTransformer 


""" gexp is boolexp grammar on top of exp grammar: 
    numeric operators have higher precedence than boolean operators.
"""
grammar_gexp = """
    ?start: boolexp
    ?boolexp: booland
        | boolexp "or" booland -> orop
    ?booland: boolnot
        | booland "and" boolnot -> andop
    ?boolnot: boolcmp
        | "not" boolnot     -> notop
    ?boolcmp: expcmp
        | boolcmp "==" expcmp -> eq
    ?expcmp: exp
        | expcmp "<=" exp      -> lte
        | expcmp "<" exp       -> lt
    ?exp: product
        | exp "+" product   -> add
        | exp "-" product   -> sub
    ?product: atom
        | product "*" atom  -> mul
        | product "/" atom  -> div
    ?atom: NUMBER           -> number
         | "-" atom         -> neg
         | "True"           -> true
         | "False"          -> false
         | NAME             -> var
         | "(" boolexp ")"
    %import common.CNAME -> NAME
    %import common.NUMBER
    %import common.WS_INLINE
    %ignore WS_INLINE
"""

In [7]:
class Tree_GExp(InlineTransformer):
    number = float

    def var(self, name):
        return ['variable', str(name)]
    
    def false(self):
        return ['bool_const', False]
    
    def true(bool_const):
        return ['bool_const', True]
    
    def orop(self, left, right):
        return ['or', left, right]
    
    def andop(self, left, right):
        return ['and', left, right]

    def notop(self, value):
        return ['not', value]

    def lte(self, left, right):
        return ['<=', left, right]

    def lt(self, left, right):
        return ['<', left, right]

    def eq(self, left, right):
        return ['==', left, right]

    def number(self, value):
        return ['number_const', float(value)]

    def add(self, left, right):
        return ['+', left, right]
    
    def sub(self, left, right):
        return ['-', left, right]
    
    def mul(self, left, right):
        return ['*', left, right]
    
    def div(self, left, right):
        return ['/', left, right]
    
    def neg(self, left):
        return ['-', left]

In [8]:
def solve_exp(exp, sigma):
    op = exp[0]
    
    """ Algebra of numbers """
    if op=='number_const':
        return exp[1]
    if op=='variable':
        return sigma[exp[1]] # Accessing the value assigned to the variable name in sigma
    if op=='+':
        return solve_exp(exp[1], sigma) + solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==3:
        return solve_exp(exp[1], sigma) - solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==2:
        return -solve_exp(exp[1], sigma)
    if op=='*':
        return solve_exp(exp[1], sigma) * solve_exp(exp[2], sigma)
    if op=='/':
        return solve_exp(exp[1], sigma) / solve_exp(exp[2], sigma)
    
    """ Algebra of Booleans """
    if op=='bool_const':
        return exp[1]
    if op=='or':
        return bool(solve_exp(exp[1], sigma)) or bool(solve_exp(exp[2], sigma))
    if op=='and':
        return bool(solve_exp(exp[1], sigma)) and bool(solve_exp(exp[2], sigma))
    if op=='not':
        return not solve_exp(exp[1], sigma)
    if op=='==':
        return solve_exp(exp[1], sigma) == solve_exp(exp[2], sigma)
    if op=='<=':
        return solve_exp(exp[1], sigma) <= solve_exp(exp[2], sigma)
    if op=='<':
        return solve_exp(exp[1], sigma) < solve_exp(exp[2], sigma)  

def get_parser_gexp():
    parser = Lark(grammar_gexp, parser='lalr', transformer=Tree_GExp())
    return parser.parse

### Examples of memory addressing

In [9]:
parse_exp = get_parser_gexp()
    
E1 = parse_exp("x + 2*y")
print(E1)

['+', ['variable', 'x'], ['*', ['number_const', 2.0], ['variable', 'y']]]


In [10]:
sigma = { 'x':0, 'y':1 } # sigma maps names to addresses, mem maps addresses to values
mem[0] = 2 
mem[1] = 3
stack_pos = 2 # Next available position on the stack
mem_dump(sigma)

# print('E1 = ', solve_exp(E1, sigma))

sigma =  {'x': 0, 'y': 1}  mem =  [2, 3, '...']


In [11]:
""" new address associated to x """
sigma = { 'x':2, 'y':1 }
mem[2] = 1
stack_pos += 1
mem_dump(sigma)

# print('E1 = ', solve_exp(E1, sigma))

sigma =  {'x': 2, 'y': 1}  mem =  [2, 3, 1, '...']


In [12]:
""" variable may share same address """
sigma = { 'x':0, 'y':0 }
mem[0] = 2
stack_pos = 1
mem_dump(sigma)

# print('E1 = ', solve_exp(E1, sigma))

sigma =  {'x': 0, 'y': 0}  mem =  [2, '...']


# Iterpreter for a C-like language

## Grammar

The new grammar builds up on the Expressions and Boolean one. 

In [13]:
from lark import Lark 

grammar_cmd = """
    ?start: cmd
    ?cmd: cmdbase
        | cmd cmdbase           -> seq
    ?cmdbase: "var" NAME ";"    -> vardecl
        | NAME "=" gexp ";"     -> assignment
        | "*" NAME "=" gexp ";" -> derefassignment
        | "{" cmd "}"       -> block
        | "if" "(" gexp ")" cmdbase "else" cmdbase -> ifelse
        | "if" "(" gexp ")" cmdbase     -> ifcond
        | "while" "(" gexp ")" cmdbase  -> loop
        | "print" "(" gexp ")" ";"      -> pprint
        | "free" "(" NAME ")" ";"       -> free
""" + grammar_gexp.replace("\n    ?start:", "    ?gexp:").\
replace("\n         | NAME", """
         | "&" NAME           -> ref
         | "*" NAME           -> deref
         | "malloc" "(" gexp ")"   -> malloc
         | NAME "(" gexp ")"  -> call
         | NAME""")

Here the complete grammar for our language

In [14]:
print(grammar_cmd)


    ?start: cmd
    ?cmd: cmdbase
        | cmd cmdbase           -> seq
    ?cmdbase: "var" NAME ";"    -> vardecl
        | NAME "=" gexp ";"     -> assignment
        | "*" NAME "=" gexp ";" -> derefassignment
        | "{" cmd "}"       -> block
        | "if" "(" gexp ")" cmdbase "else" cmdbase -> ifelse
        | "if" "(" gexp ")" cmdbase     -> ifcond
        | "while" "(" gexp ")" cmdbase  -> loop
        | "print" "(" gexp ")" ";"      -> pprint
        | "free" "(" NAME ")" ";"       -> free
    ?gexp: boolexp
    ?boolexp: booland
        | boolexp "or" booland -> orop
    ?booland: boolnot
        | booland "and" boolnot -> andop
    ?boolnot: boolcmp
        | "not" boolnot     -> notop
    ?boolcmp: expcmp
        | boolcmp "==" expcmp -> eq
    ?expcmp: exp
        | expcmp "<=" exp      -> lte
        | expcmp "<" exp       -> lt
    ?exp: product
        | exp "+" product   -> add
        | exp "-" product   -> sub
    ?product: atom
        | product "*" atom  -> mul

## Parser

In [15]:
class Tree_Cmd(Tree_GExp): # Extends the previously defined parser for expressions and booleans
    
    def vardecl(self, var):
        return ['decl', str(var)]

    def assignment(self, var, expr):
        return ['=', str(var), expr]

    def derefassignment(self, var, expr):
        return ['deref=', str(var), expr]

    def seq(self, cmd1, cmd2):
        if cmd1[0]=='seq':
            return ['seq'] + cmd1[1:] + [cmd2]
        return ['seq', cmd1, cmd2]

    def block(self, cmd):
        return ['block', cmd]

    def ifelse(self, cond, cmdif, cmdelse):
        return ['if', cond, cmdif, cmdelse]
    
    def ifcond(self, cond, cmd):
        return ['if', cond, cmd]

    def loop(self, cond, cmd):
        return ['while', cond, cmd]

    def pprint(self, gexp):
        return ['print', gexp]

    def ref(self, var):
        return ['ref', str(var)]

    def deref(self, var):
        return ['deref', str(var)]

    def malloc(self, exp):
        return ['malloc', exp]

    def free(self, var):
        return ['free', str(var)]

In [16]:
def get_parser_cmd():
    parser = Lark(grammar_cmd, parser='lalr', transformer=Tree_Cmd())
    return parser.parse 

### Example

In [17]:
parse = get_parser_cmd()
    
C1 = parse("var x; x = 2; var y; y = 1; var z; z = x+2*y; print(z);")
C1    

['seq',
 ['decl', 'x'],
 ['=', 'x', ['number_const', 2.0]],
 ['decl', 'y'],
 ['=', 'y', ['number_const', 1.0]],
 ['decl', 'z'],
 ['=',
  'z',
  ['+', ['variable', 'x'], ['*', ['number_const', 2.0], ['variable', 'y']]]],
 ['print', ['variable', 'z']]]

## Interpreter

- Expression & Boolean interpreter extended with memory addressing (and a few new operators...)

In [18]:
def solve_exp(exp, sigma):
    op = exp[0]
    
    """ Algebra of numbers """
    if op=='number_const':
        return exp[1]
    if op=='variable':
        return mem[sigma[exp[1]]] # Memory addressing!
    if op=='+':
        return solve_exp(exp[1], sigma) + solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==3:
        return solve_exp(exp[1], sigma) - solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==2:
        return -solve_exp(exp[1], sigma)
    if op=='*':
        return solve_exp(exp[1], sigma) * solve_exp(exp[2], sigma)
    if op=='/':
        return solve_exp(exp[1], sigma) / solve_exp(exp[2], sigma)
    
    """ Algebra of Booleans """
    if op=='bool_const':
        return exp[1]
    if op=='or':
        return bool(solve_exp(exp[1], sigma)) or bool(solve_exp(exp[2], sigma))
    if op=='and':
        return bool(solve_exp(exp[1], sigma)) and bool(solve_exp(exp[2], sigma))
    if op=='not':
        return not solve_exp(exp[1], sigma)
    if op=='==':
        return solve_exp(exp[1], sigma) == solve_exp(exp[2], sigma)
    if op=='<=':
        return solve_exp(exp[1], sigma) <= solve_exp(exp[2], sigma)
    if op=='<':
        return solve_exp(exp[1], sigma) < solve_exp(exp[2], sigma)
    
    # new operators
    if op=='ref':
        return sigma[exp[1]]
    if op=='deref':
        return mem[mem[sigma[exp[1]]]] 
    if op=='malloc':
        return malloc(int(solve_exp(exp[1], sigma)))

Support function for updating the sigma

In [19]:
def merge_two_dicts(d1, d2):
    d = d1.copy()
    d.update(d2)
    return d

- Interpreter for C-like commands

In [20]:
def solve_cmd(cmd, sigma):
    
    global stack_pos
    op = cmd[0]
    
    if op=='decl':
        var = cmd[1]
        """ stack pushing """
        address = stack_pos
        stack_pos += 1
        return merge_two_dicts(sigma, {var:address})
    
    if op=='=':
        var = cmd[1]
        expr = cmd[2]
        mem[sigma[var]] = solve_exp(expr, sigma)
        return sigma
    
    if op=='seq':
        for c in cmd[1:]:
            sigma = solve_cmd(c, sigma)
        return sigma
    
    if op=='print':
        val = solve_exp(cmd[1], sigma)
        print(val)
        return sigma
    
    if op=='block':
        tmp = stack_pos
        solve_cmd(cmd[1], sigma)
        """ stack popping """
        stack_pos = tmp
        return sigma
    
    if op=='if':
        b = solve_exp(cmd[1], sigma)
        if b:
            return solve_cmd(cmd[2], sigma)
        if len(cmd)==4: #if else
            return solve_cmd(cmd[3], sigma)
        return sigma
    
    if op=='while':
        b = solve_exp(cmd[1], sigma)
        if not b:
            return sigma
        body = cmd[2]
        sigma = solve_cmd(body, sigma)
        return solve_cmd(cmd, sigma)
    
    if op=='deref=':
        var = cmd[1]
        expr = cmd[2]
        mem[int(mem[sigma[var]])] = solve_exp(expr, sigma)
        return sigma
    
    if op=='free':
        var = cmd[1]
        free(mem[sigma[var]])
        mem[sigma[var]] = None
        return sigma

### Examples of memory addressing (cont'd)

Let's evaluate the previous expression while varying (sigma, mem)

In [21]:
parse_exp = get_parser_gexp()
    
E1 = parse_exp("x + 2*y")
print(E1)

['+', ['variable', 'x'], ['*', ['number_const', 2.0], ['variable', 'y']]]


In [22]:
sigma = { 'x':0, 'y':1 } # sigma maps names to addresses, mem maps addresses to values
mem[0] = 2 
mem[1] = 3
stack_pos = 2 # Next available position on the stack
mem_dump(sigma)

print('E1 = ', solve_exp(E1, sigma))

sigma =  {'x': 0, 'y': 1}  mem =  [2, 3, '...']
E1 =  8.0


In [23]:
""" new address associated to x """
sigma = { 'x':2, 'y':1 }
mem[2] = 1
stack_pos += 1
mem_dump(sigma)

print('E1 = ', solve_exp(E1, sigma))

sigma =  {'x': 2, 'y': 1}  mem =  [2, 3, 1, '...']
E1 =  7.0


In [24]:
""" variable may share same address """
sigma = { 'x':0, 'y':0 }
mem[0] = 2
stack_pos = 1
mem_dump(sigma)

print('E1 = ', solve_exp(E1, sigma))

sigma =  {'x': 0, 'y': 0}  mem =  [2, '...']
E1 =  6.0


## Examples: Commands 

Run the C code at http://pythontutor.com/c.html#mode=edit

In [25]:
parse = get_parser_cmd()

In [26]:
C1 = parse("var x; x = 2; var y; y = 1; var z; z = x+2*y; print(z);")
sigma = {}
mem_init()
sigmares = solve_cmd(C1, sigma)

print("\nmem_dump:")
mem_dump(sigmares)

c_code = """ 
float x;
x = 2;
float y;
y = 1;
float z;
z = x + 2*y;
"""

4.0

mem_dump:
sigma =  {'x': 0, 'y': 1, 'z': 2}  mem =  [2.0, 1.0, 4.0, '...']


In [27]:
C2 = parse("{ var x; x = 2; var y; y = 1; var z; z = x+2*y; print(z);}")
mem_init()
sigmares = solve_cmd(C2, sigma)

print("\nmem_dump:")
mem_dump(sigmares)

c_code = """ 
{
    float x;
    x = 2;
    float y;
    y = 1;
    float z;
    z = x + 2*y;
}
"""

4.0

mem_dump:
sigma =  {}  mem =  ['...']


In [28]:
C3 = parse("{ var x; x = 2; {var y; y = 1; var z; z = 2*y; print(z);} }")
mem_init()
sigmares = solve_cmd(C3, sigma)

print("\nmem_dump:")
mem_dump(sigmares)

c_code = """ C-code
float x;
x = 2;
{
    float y;
    y = 1;
    float z;
    z = 2*y;
}
"""

2.0

mem_dump:
sigma =  {}  mem =  ['...']


In [29]:
C4 = parse("var i; i=5; while(0<i) { print(i); i = i-1; }")
mem_init()
sigmares = solve_cmd(C4, sigma)

print("\nmem_dump:")
mem_dump(sigmares)

c_code = """ C-code
int i;
i = 5;
while(0<i)
{
    printf("%d\n", i);
    i--;
}
"""

5.0
4.0
3.0
2.0
1.0

mem_dump:
sigma =  {'i': 0}  mem =  [0.0, '...']


### Examples: Pointers (Heap)

In [30]:
C5 = parse("var x; x=3; var y; y = &x; *y = *y + 1; print(x);")
mem_init()
sigmares = solve_cmd(C5, sigma)

print("\nmem_dump:")
mem_dump(sigmares)

c_code = """ C-code
float x;
x = 3;
float *y;
y = &x;
*y = *y + 1;
printf("%f\n", x);
"""

4.0

mem_dump:
sigma =  {'x': 0, 'y': 1}  mem =  [4.0, 0, '...']


In [31]:
C6 = parse("var x; x=3; var y; y = malloc(1); *y = x + 1; print(*y); print(y); ")
mem_init()
sigmares = solve_cmd(C6, sigma)

print("\nmem_dump:")
mem_dump(sigmares)

c_code = """ C-code
float x;
x = 3;
float *y;
y = malloc(sizeof(float));
*y = x + 1;
printf("%f %p\n", *y, y);
free(y);
"""

4.0
999

mem_dump:
sigma =  {'x': 0, 'y': 1}  mem =  [3.0, 999, '...', 4.0]


In [32]:
C7 = parse("""
    var i; 
    i=5; 
    var y; 
    y = malloc(i); 
    var x; 
    x = 5; 
    while(0<i) 
    { 
        i = i-1;
        var z;
        z = y+i;
        *z = x;
        x = x-1;
    }""".replace("\n",""))

mem_init()
sigmares = solve_cmd(C7, sigma)
mem_dump(sigmares)

c_code = """ C-code
int i;
i = 5;
float *y;
y = malloc(sizeof(float)*i);
float x;
x=5;
while(0<i)
{
    i--;
    *(y+i) = x;
    x--;
}
"""

sigma =  {'i': 0, 'y': 1, 'x': 2}  mem =  [0.0, 995, 0.0, '...', 1.0, 2.0, 3.0, 4.0, 5.0]
